In [324]:
#Import library
import ipywidgets as widgets
from IPython.display import HTML

from re import finditer

## Simple color management

In [340]:
import random
import math
import colorsys

myColors={} 

def colorFor(hashCode):
    if hashCode in myColors:
        return myColors[hashCode]
    else:
        hue = random.random()
        sat = random.random()
        rgb = colorsys.hsv_to_rgb(hue, sat, 0.9)
        r = int(rgb[0] * 255)
        g = int(rgb[1] * 255)
        b = int(rgb[2] * 255)
        newCol = (r,g,b)
        myColors[hashCode] = newCol
        return newCol

def hexColorFor(rgb):
    opacityShade = 0.3
    return 'rgba(%d,%d,%d,%f)' % (rgb[0], rgb[1], rgb[2], opacityShade)
    
def meanColorFor(colorArr):
    r=0
    g=0
    b=0
    for color in colorArr:
        r += color[0]
        g += color[1]
        b += color[2]
        
    arrLen = len(colorArr)
    return (int(r/arrLen), int(g/arrLen), int(b/arrLen))  

def debugFormatColor(color, name):
    return  str(color) + "=<b style=\"background-color:" + hexColorFor(color) + "\">" + name + "</b><br/>"


colorA = (255,0,0)
colorB = (0,0,255)
combined = [colorA, colorB]
mean = meanColorFor(combined)

colorC = (0,255,0)
mean2 = meanColorFor([colorA, colorB, colorC])

widgetStr = ""
widgetStr += debugFormatColor(colorA, "colorA")
widgetStr += debugFormatColor(colorB, "colorB")
widgetStr += debugFormatColor(mean, "mean of A & B")
widgetStr += debugFormatColor(colorC, "colorC")
widgetStr += debugFormatColor(mean2, "mean of A, B, C")
widgetStr += debugFormatColor(meanColorFor([colorA, colorC]), "mean of A, C")
widgetStr += debugFormatColor(meanColorFor([colorB, colorC]), "mean of B, C")

widgets.HTML(
    value=widgetStr
)




HTML(value='(255, 0, 0)=<b style="background-color:rgba(255,0,0,0.300000)">colorA</b><br/>(0, 0, 255)=<b style…

## Class structures
Let's experiment with a self-aware token object

In [326]:
class HighlightedFile():
    def __init__(self,filename):
        self.filename = filename
    def lines(self):
        with open(self.filename, 'r') as file:
            sampleLines = file.read()       
        # make the char index the correct length
        self.chars = [None] * len(sampleLines)
        
        # initialise the char index
        charCtr = 0
        for char in sampleLines:
            # put letter into a struct
            charInd = CharIndex(char)
            chars[charCtr] = charInd
            charCtr += 1         
        
        # ok, break the file into self-aware lines
        lineCtr = 0
        lines = []
        strLines = sampleLines.splitlines()
        for thisLine in strLines:
            thisLen = len(thisLine)
            newL = Line(str(lineCtr), str(lineCtr + thisLen), thisLine, chars)
            lines.append(newL)
            lineCtr += thisLen + 1
            
        return lines
    def export(self, filename):
        fOut = open(filename, "w")

        lastHash = ""

        for charIndex in chars:
            letter = charIndex.letter
            thisHash = ""
            thisMessage = ""
            colors = []
            for usage in charIndex.usages:
                thisHash += usage.toolField
                needsNewLine = thisMessage != ""
                colors.append(colorFor(usage.toolField))
                if needsNewLine:
                    thisMessage += " // "
                thisMessage += usage.toolField + ", " + usage.message

            # do we have anything to shade?
            if thisHash != "":
                # generate/retrieve a color for this hash
                newColor = meanColorFor(colors)
                hexColor = hexColorFor(newColor)

                # are we already in hash?
                if lastHash != "":
                    # is it the different to this one?
                    if(lastHash != thisHash):
                        # ok, close the span
                        fOut.write("</span>")

                        # start a new span
                        fOut.write("<span title='"+thisMessage+"' style=\"background-color:" + hexColor + "\"a>")
                else:
                    fOut.write("<span title='"+thisMessage+"' style=\"background-color:" + hexColor + "\">")
            elif lastHash != "":
                fOut.write("</span>")

            # just check if it's newline
            if(letter == "\n"):
                fOut.write("</br>")
            else:
                fOut.write(letter)

            lastHash = thisHash

        if(lastHash != ""):
            fOut.write("</span>")

        fOut.close()     

WHITESPACE_DELIM = "\\S+"
CSV_DELIM = "(?:,\"|^\")(\"\"|[\w\W]*?)(?=\",|\"$)|(?:,(?!\")|^(?!\"))([^,]*?)(?=$|,)|(\r\n|\n)"

# storage structure used to describe the activity on a single character    
class CharIndex:
    def __init__(self, letter):
        self.letter = letter
        self.usages = []
    def __str__(self):
        message = "["+ self.letter+"]"
        for usage in self.usages:
            message+= "(T/F:" + usage.toolField + ", msg:" + usage.message + ")"
        return message

# storage of a single activity    
class SingleUsage:
    def __init__(self, toolField, message):
        self.toolField = toolField
        self.message = message
    

# a line from the file
class Line():
    def __init__(self,start, end,text, chars):
        self.start=start
        self.end=end
        self.text=text
        self.chars = chars
    def tokens(self, regExp=WHITESPACE_DELIM, stripChar=""):
        tokens = []
        for match in finditer(regExp, self.text):
            tokenStr = match.group()
            # special handling, we may need to strip a leading delimiter
            if stripChar != "":
               charIndex = tokenStr.find(stripChar)
               if charIndex == 0 :
                   tokenStr = tokenStr[1:] 
               # and ditch any new whitespace
               tokenStr = tokenStr.strip()
               
            tokens.append(Token(match.span(), tokenStr, int(self.start), self.chars))
        return tokens
    def record(self, tool, message):
        for i in range(int(self.start), int(self.end)):
            usage = SingleUsage(tool, message)
            self.chars[i].usages.append(usage)

class Token():
    def __init__(self,span,text, lineStart, chars):
        self.span=span
        self.text=text
        self.lineStart = lineStart
        self.chars = chars
    def __str__(self):
        return "[("+ str(self.start())+"-"+str(self.end())+")"+":\""+self.text+"\"]"
    def start(self):
        return self.lineStart + self.span[0]
    def end(self):
        return self.lineStart + self.span[1]
    def record(self, tool, field, value, units):
        toolField = tool+"/"+field
        message = "Value:" + str(value) + " Units:" + str(units)        
        for i in range(self.start(), self.end()):
            usage = SingleUsage(toolField, message)
            self.chars[i].usages.append(usage)

## Try to import a text file

In [327]:
text_file = open("file.txt")
file_content = text_file.read()
print(file_content)
text_file.close()

951212 050000.000 MONDEO_44   @C   269.7   2.0      10
// EVENT 951212 050300.000 BRAVO
// EVENT 951212 050300.000 CHARLIE
951212 050300.000 FORD_11   @C   354.7   2.1      14
951212 050200.000 COROLLA_44   @C   177.9   3.1      15
// EVENT 951212 050300.000 DELTA
951212 050300.000 COROLLA_44   @C   200   3.1      15



## Try out the simple event importer

In [328]:
class EventImporter():
    def importThese(self, lines):
        # process the lines
        for thisLine in lines:
            tokens = thisLine.tokens()

            # check the type
            firstToken = tokens[0]
            if firstToken.text == "//":
                # event marker
                eventImporter = "Simple Event importer"
                dateToken = tokens[2]
                dateToken.record(eventImporter,"Date", dateToken.text, "n/a")
                timeToken = tokens[3]
                timeToken.record(eventImporter,"Time", timeToken.text, "n/a")
                eventToken = tokens[4]
                eventToken.record(eventImporter,"Event", timeToken.text, "n/a")

                # and the whole=line record
                thisLine.record(eventImporter, "Whole line")

In [329]:
# create the self-highlighter
dataFile = HighlightedFile('file.txt')

# get the set of self-describing lines
lines = dataFile.lines()

# get an importer
importer = EventImporter()

# do the import
importer.importThese(lines)

# output to file, display
dataFile.export("out4.html")
HTML(filename="./out4.html")

## Try a State import tool

In [330]:
class StateImporter():
    def importThese(self, lines):
        # process the lines
        for thisLine in lines:
            tokens = thisLine.tokens()
            myName = "State importer"
            
            # check the type
            firstToken = tokens[0]
            if firstToken.text != "//":
                dateToken = tokens[0]
                dateToken.record(myName,"Date", dateToken.text, "n/a")

                timeToken = tokens[1]
                timeToken.record(myName,"Time", timeToken.text, "n/a")

                vehicleToken = tokens[2]
                vehicleVal = vehicleToken.text
                vehicleToken.record(myName,"Vehicle", vehicleVal,"n/a")  

                directionToken = tokens[4]
                directionVal = float(directionToken.text)
                directionToken.record(myName,"Direction", directionVal,"degs")  

                speedToken = tokens[5]
                speedVal = float(speedToken.text)
                speedToken.record(myName,"Speed", speedVal,"m/s")  

                speedToken = tokens[5]
                speedVal = float(speedToken.text)
                speedToken.record(myName,"Other Speed", speedVal,"m/s")  

                dirToken = tokens[6]
                directionVal = float(dirToken.text)
                dirToken.record(myName,"Direction", directionVal,"degs")

In [331]:
# create the self-highlighter
dataFile = HighlightedFile('file.txt')

# get the set of self-describing lines
lines = dataFile.lines()

# get an importer
importer = StateImporter()

# do the import
importer.importThese(lines)

# output to file, display
dataFile.export("out4.html")
HTML(filename="./out4.html")

In [332]:
# create the self-highlighter
dataFile = HighlightedFile('file.txt')

# get the set of self-describing lines
lines = dataFile.lines()

# handle the first importer
importer = EventImporter()
importer.importThese(lines)

# now the second importer
stateImporter = StateImporter()
stateImporter.importThese(lines)

# output to file, display
dataFile.export("out4.html")
HTML(filename="./out4.html")

## Try a composite importer

In [333]:
class CompositeImporter():
    def importThese(self, lines):
        # process the lines
        for thisLine in lines:
            tokens = thisLine.tokens()
            myName = "Composite Importer"

            # check the type
            firstToken = tokens[0]
            if firstToken.text == "//":
                # event marker
                eventImporter = "Composite Importer (Event)"
                dateToken = tokens[2]
                dateToken.record(eventImporter,"Date", dateToken.text, "n/a")
                timeToken = tokens[3]
                timeToken.record(eventImporter,"Time", timeToken.text, "n/a")
                eventToken = tokens[4]
                eventToken.record(eventImporter,"Event", timeToken.text, "n/a")

                # and the whole=line record
                thisLine.record(eventImporter, "Whole line")

            else:

                dateToken = tokens[0]
                dateToken.record(myName,"Date", dateToken.text, "n/a")

                timeToken = tokens[1]
                timeToken.record(myName,"Time", timeToken.text, "n/a")

                vehicleToken = tokens[2]
                vehicleVal = vehicleToken.text
                vehicleToken.record(myName,"Vehicle", vehicleVal,"n/a")  

                directionToken = tokens[4]
                directionVal = float(directionToken.text)
                directionToken.record(myName,"Direction", directionVal,"degs")  

                speedToken = tokens[5]
                speedVal = float(speedToken.text)
                speedToken.record(myName,"Speed", speedVal,"m/s")  

                speedToken = tokens[5]
                speedVal = float(speedToken.text)
                speedToken.record(myName,"Other Speed", speedVal,"m/s")  

                dirToken = tokens[6]
                directionVal = float(dirToken.text)
                dirToken.record(myName,"Direction", directionVal,"degs")

In [334]:
# create the self-highlighter
dataFile = HighlightedFile('file.txt')

# get the set of self-describing lines
lines = dataFile.lines()

# get an importer
importer = CompositeImporter()

# do the import
importer.importThese(lines)

# output to file, display
dataFile.export("out4.html")
HTML(filename="./out4.html")

## Try out comma-delimited file

In [335]:
text_file = open("file_comma.txt")
file_content = text_file.read()
print(file_content)
text_file.close()

951212, 050000.000, MONDEO_44, @C, 269.7, 2.0, 10
//, EVENT, 951212, 050300.000, BRAVO
//, EVENT, 951212, 050300.000, CHARLIE
951212, 050300.000, FORD_11, @C, 354.7, 2.1, 14
951212, 050200.000, COROLLA_44, @C, 177.9, 3.1, 15
//, EVENT, 951212, 050300.000, DELTA
951212, 050300.000, COROLLA_44, @C, 200, 3.1, 15



In [336]:
class CSVEventImporter():
    def importThese(self, lines):
        # process the lines
        for thisLine in lines:
            
            
            tokens = thisLine.tokens(CSV_DELIM, ",") # note we specify delimiter

            #print(tokens[0], " ", tokens[1], " ", tokens[2], " ", tokens[3]) 
            
            # check the type
            firstToken = tokens[0]
                  
            if firstToken.text == "//":
                # event marker
                eventImporter = "Simple CSV Event importer"
                dateToken = tokens[2]
                dateToken.record(eventImporter,"Date", dateToken.text, "n/a")
                timeToken = tokens[3]
                timeToken.record(eventImporter,"Time", timeToken.text, "n/a")
                eventToken = tokens[4]
                eventToken.record(eventImporter,"Event", timeToken.text, "n/a")

                # and the whole=line record
                thisLine.record(eventImporter, "Whole line")

In [337]:
# create the self-highlighter
dataFile = HighlightedFile('file_comma.txt')

# get the set of self-describing lines
lines = dataFile.lines()

# get an importer
importer = CSVEventImporter()

# do the import
importer.importThese(lines)

# output to file, display
dataFile.export("out4.html")
HTML(filename="./out4.html")